# Robot Web Controller

Here's a step-by-step guide to create a web server container on your Raspberry Pi, designed to evolve into a full-stack IoT/robot control system:

## 📂 Project Structure

```
~/docker/robot_web_controller/
                ├── docker-compose.yml
                ├── web/
                │   ├── Dockerfile
                │   ├── html/
                │   │   └── index.html
                │   └── cgi-bin/
                │       ├── ledon.py
                │       └── ledoff.py
                └── scripts/
                    └── camera_stream.py
```

```bash
mkdir ~/docker/robot_web_controller
cd ~/docker/robot_web_controller
mkdir -p web/{html,cgi-bin}
```

## 🐋 Step 1: Create docker-compose.yml

`nano docker-compose.yml`
```yaml
version: '3.8'

services:
  web_server:
    build: ./web # Points to folder with Dockerfile
    container_name: robot_web
    devices:
      - "/dev/gpiomem:/dev/gpiomem"
      - "/dev/mem:/dev/mem"
    privileged: true  # Required for GPIO
    ports:
      - "83:80"
      - "8000:8000"  # For future WebSocket
    volumes:
      - ./web/html:/usr/local/apache2/htdocs
      - ./web/cgi-bin:/usr/local/apache2/cgi-bin
      - /sys/class/gpio:/sys/class/gpio
    restart: unless-stopped

  camera_service:  # Will add later
    image: python:3.9-slim
    # We'll expand this after testing web server

  # Add more services like database, WebSocket, etc. later
```
## Step 2: Create a Dockerfile with no extention: 
`nano web/Dockerfile`

```dockerfile
FROM httpd:2.4-alpine

# 1. Install system Python and required dependencies
RUN apk update && apk add --no-cache \
    bash \
    nano \
    python3 \
    py3-pip \
    python3-dev \
    gcc \
    musl-dev \
    linux-headers

# 2. Create and activate virtual environment
RUN python3 -m venv /opt/venv
ENV PATH="/opt/venv/bin:$PATH"

# 3. Install RPi.GPIO in virtual environment
RUN pip install --no-cache-dir RPi.GPIO

# 4. Clean up build dependencies
RUN apk del python3-dev gcc musl-dev linux-headers

# 5. Configure Apache for CGI
RUN sed -i \
    -e 's/#LoadModule cgid_module/LoadModule cgid_module/' \
    -e 's/#AddHandler cgi-script .cgi/AddHandler cgi-script .cgi .py/' \
    /usr/local/apache2/conf/httpd.conf

# 6. Configure Apache to run as root
RUN sed -i \
    -e 's/User daemon/User root/' \
    -e 's/Group daemon/Group root/' \
    /usr/local/apache2/conf/httpd.conf
    
# 7. Ensure scripts use the virtual environment Python
RUN find /usr/local/apache2/cgi-bin -name '*.py' -exec sed -i '1s|^.*$|#!/opt/venv/bin/python3|' {} \;

# 8. Keep your existing configurations
RUN echo "ServerName localhost" >> /usr/local/apache2/conf/httpd.conf

# 9. Clean up build dependencies (optional)
RUN apk del python3-dev gcc musl-dev linux-headers
```

## 🌐 Step 3: Create a Test Webpage

`nano web/html/index.html`

Minimal test page:
```html
<!DOCTYPE html>
<html>
<head>
    <title>My Robot</title>
    <style>
        body { font-family: sans-serif; text-align: center; margin-top: 50px; }
        button { padding: 10px 20px; font-size: 16px; }
    </style>
</head>
<body>
    <h1>Hello from Docker!</h1>
    <p>My Raspberry Pi web server is running 🎉</p>
    <button onclick="alert('Future robot control!')">Test Button</button>
</body>
</html>
```
[Enhanced_LED_Control_Interface HTML](Enhanced_LED_Control_Interface.ipynb)

## 🚀 Step 4: Start Your Container

Run this command from your project folder:
`docker compose up -d`


## 🔍 Step 5: Verify It Works

1. Find your Raspberry Pi's IP address:

```bash
hostname -I
```
2. Open a web browser on any device in your network and visit:

```text
[YOUR_PI_IP]:83
```
You should see your test webpage with the button!

## 🔄 Workflow for Development

1. Edit files locally in ~/robot_web_controller/web/
2. Changes appear immediately (thanks to volume mounts)
3. For Dockerfile changes, rebuild:
   ```bash
    docker-compose up -d --build
   ```

## ➡️ Next Steps (Ready When You Are):

1. Add WebSocket Service: Python server using websockets library
2. Camera Streaming: OpenCV container with MJPEG stream
3. Database: Add a PostgreSQL container for logging
4. Authentication: Basic auth for web interface

---
## 👨🏻‍💻 Next Phase - Test cgi-bin python scripts
### 1. Create an `ledon.py` script
`nano web/cgi-bin/ledon.py`

```python
#!/opt/venv/bin/python3

print("Content-Type: text/html\n")
print("<html><body><h1>Hello from Python CGI! LED ON</h1></body></html>")
```
- Make sure it is executable: `sudo chmod +x web/cgi-bin/*.py`
- Ensure scripts use the virtual environment Python:
`docker exec robot_web find /usr/local/apache2/cgi-bin -name '*.py' -exec sed -i '1s|^.*$|#!/opt/venv/bin/python3|' {} \;`

### 2. Update the HTML code
`nano web/html/index.html`

Test page for cgi-bin script:
```html
<!DOCTYPE html>
<html>
<head>
    <title>LED Control Test</title>
    <style>
        body { font-family: sans-serif; text-align: center; margin-top: 50px; }
        button { 
            padding: 15px 30px; 
            font-size: 18px;
            margin: 10px;
            background: #4CAF50;
            color: white;
            border: none;
            border-radius: 5px;
            cursor: pointer;
        }
        #response { margin-top: 20px; min-height: 50px; }
    </style>
</head>
<body>
    <h1>LED Control Test</h1>
    
    <!-- Method 1: Simple link -->
    <a href="/cgi-bin/ledon.py" style="display: block; margin: 20px;">
        <button>Test LED ON (Direct Link)</button>
    </a>
    
    <!-- Method 2: AJAX version -->
    <button onclick="testLedOn()">Test LED ON (AJAX)</button>
    <div id="response"></div>

    <script>
        function testLedOn() {
            const responseDiv = document.getElementById('response');
            responseDiv.innerHTML = "Sending request...";
            
            fetch('/cgi-bin/ledon.py')
                .then(response => response.text())
                .then(data => {
                    responseDiv.innerHTML = data;
                })
                .catch(error => {
                    responseDiv.innerHTML = `Error: ${error}`;
                });
        }
    </script>
</body>
</html>
```
Here is an [Enhanced_LED_Control_Interface HTML](Enhanced_LED_Control_Interface.ipynb)

### 3. 🧪 Test Your Setup

Direct Browser Test, Access your page:
```text
[YOUR_PI_IP]:83
```

---
## ⏭️ Next Phase - GPIO control

### 1. Update the docker-compose.yml
`nano docker-compose.yml`

```yml
services:
  web_server:
    build: ./web
    container_name: robot_web
    privileged: true  # Full hardware access
    user: root  # Force container to run as root
    ports:
      - "83:80"
    volumes:
      - ./web/html:/usr/local/apache2/htdocs
      - ./web/cgi-bin:/usr/local/apache2/cgi-bin
    restart: unless-stopped
```

### 2. Update the Dockerfile
`nano web/Dockerfile`

```dockerfile
FROM httpd:2.4

# Install dependencies
RUN apt-get update && apt-get install -y \
    python3 \
    python3-rpi.gpio \
    sudo \
    && rm -rf /var/lib/apt/lists/*

# Configure Apache to run CGI scripts as root
RUN echo "Defaults:www-data !requiretty" >> /etc/sudoers && \
    echo "www-data ALL=(ALL) NOPASSWD: /usr/bin/python3" >> /etc/sudoers && \
    chmod 440 /etc/sudoers

# Enable CGI
RUN sed -i \
    -e 's/#LoadModule cgid_module/LoadModule cgid_module/' \
    -e 's/#AddHandler cgi-script .cgi/AddHandler cgi-script .cgi .py/' \
    /usr/local/apache2/conf/httpd.conf

# Create wrapper script
RUN echo '#!/bin/sh\nsudo /usr/bin/python3 "$@"' > /usr/local/bin/python3-cgi && \
    chmod +x /usr/local/bin/python3-cgi
```

### 3. Update `ledon.py` script
`nano web/cgi-bin/ledon.py`

```python
#!/usr/local/bin/python3-cgi
print("Content-type: text/plain\n\n")

try:
    import RPi.GPIO as GPIO
    GPIO.setwarnings(False)
    GPIO.setmode(GPIO.BCM)
    GPIO.setup(17, GPIO.OUT)
    GPIO.output(17, GPIO.HIGH)
    print("SUCCESS: LED ON - GPIO17 HIGH")
except Exception as e:
    print(f"ERROR: {str(e)}")

# Add status feedback
GPIO.setup(17, GPIO.IN)
state = GPIO.input(17)
print(f"\nGPIO17 state: {state}")
```

### 4. Test

```bash
docker compose down
docker compose up -d --build
curl http://localhost:83/cgi-bin/ledon.py
```

---
# Running WebSocket Server on Container Start (Docker + Apache)

This guide explains how to automatically start your `led_ws_server.py` WebSocket server when your Docker container launches, alongside Apache.

## 🧱 Project Structure

Assume you have the following:

```

~/docker/robot_web_controller/
                ├── docker-compose.yml
                ├── web/
                │   ├── Dockerfile
                │   ├── supervisord.conf
                │   ├── html/
                │   │   └── index.html
                │   └── cgi-bin/
                │       ├── ledon.py
                │       ├── ledoff.py
                │       └── led_ws_server.py
                └── scripts/
                    └── camera_stream.py
 ```

---

## ✅ Step-by-Step Instructions

Add a port to the container, this is the web socket serving port
   `nano docker-compose.yml`
   - "8787:8787"

```yml
services:
  web_server:
    build: ./web
    container_name: robot_web
    privileged: true  # Full hardware access
    user: root  # Force container to run as root
    ports:
      - "83:80"
      - "8787:8787"
    volumes:
      - ./web/html:/usr/local/apache2/htdocs
      - ./web/cgi-bin:/usr/local/apache2/cgi-bin
    restart: unless-stopped
```

### 1. Install `supervisor` in your Dockerfile

Update your Dockerfile to include Supervisor:

```Dockerfile
FROM httpd:2.4

# Install necessary packages
RUN apt-get update && apt-get install -y \
    python3 python3-pip python3-venv python3-dev \
    libffi-dev build-essential sudo supervisor \
    && rm -rf /var/lib/apt/lists/*

# Set up sudo permissions
RUN echo "Defaults:www-data !requiretty" >> /etc/sudoers && \
    echo "www-data ALL=(ALL) NOPASSWD: /usr/bin/python3" >> /etc/sudoers && \
    chmod 440 /etc/sudoers

# Enable CGI
RUN sed -i \
    -e 's/#LoadModule cgid_module/LoadModule cgid_module/' \
    -e 's/#AddHandler cgi-script .cgi/AddHandler cgi-script .cgi .py/' \
    /usr/local/apache2/conf/httpd.conf

# Set up python CGI wrapper
RUN echo '#!/bin/sh\nsudo /usr/bin/python3 "$@"' > /usr/local/bin/python3-cgi && \
    chmod +x /usr/local/bin/python3-cgi

# Create virtual environment
RUN python3 -m venv /opt/venv

# Install Python packages
RUN /opt/venv/bin/pip install websockets RPi.GPIO

# Copy the supervisor config
COPY supervisord.conf /etc/supervisord.conf

# Start both services using supervisord
CMD ["/usr/bin/supervisord", "-c", "/etc/supervisord.conf"]
```

---

### 2. Create `supervisord.conf`

This configuration runs both Apache and the WebSocket server:
`nano ~/docker/robot_web_controller/supervisord.conf`

```ini
[supervisord]
nodaemon=true

[program:httpd]
command=/usr/local/bin/httpd-foreground
autorestart=true
stdout_logfile=/dev/stdout
stderr_logfile=/dev/stderr

[program:websocket]
command=/opt/venv/bin/python3 /usr/local/apache2/cgi-bin/led_ws_server.py
autorestart=true
stdout_logfile=/dev/stdout
stderr_logfile=/dev/stderr
```

---

### 3. Build and Run

Rebuild your container with no cache:

```bash
docker compose build --no-cache
```

Run it:

```bash
docker compose up -d
```

Then check logs:

```bash
docker logs -f robot_web
```

You should see:

```
WebSocket server running on ws://0.0.0.0:8787
```

---

## 🧪 Test

Visit your control panel (e.g., http://192.168.1.189:83) — that's where:

- The HTML + JS page is hosted.
- JavaScript inside it uses:
- let socket = new WebSocket("ws://192.168.1.189:8787");
- That JavaScript sends commands like LEDon, LEDoff, forward, etc.

---

## ✅ Done!

Your WebSocket server now runs automatically every time your container starts, without needing manual launching.



---
## Camera config for motioneye login

Get the apache config file from the container:
`docker cp robot_web://usr/local/apache2/conf/httpd.conf ./`
`nano httpd.conf`

Apache modules are enabled (manually via `httpd.conf`):
`LoadModule proxy_module modules/mod_proxy.so`
`LoadModule proxy_http_module modules/mod_proxy_http.so`

add the following lines to the end:

```bash
ProxyRequests Off
<Proxy *>
    Require all granted
</Proxy>

ProxyPass /camera http://host.docker.internal:8082/
ProxyPassReverse /camera http://host.docker.internal:8082/
```

Copy the config file back to the container:
`docker cp httpd.conf robot_web://usr/local/apache2/conf/httpd.conf`
`docker restart robot_web`

---

 To add the ProxyPass configuration in your Apache container, you'll need to:
 

---
<div style="font-family: Arial, sans-serif; line-height: 1.5;">
  <strong>János Rostás & ChatGPT</strong><br>
  👨‍💻 Electronic & Computer Engineer | 🛠️ Tinkerer with a Purpose | 🐳 Docker Enthusiast<br>
  🌐 <a href="https://janosrostas.co.uk" target="_blank">janosrostas.co.uk</a>
</div>
  